In [12]:
# 若无法解析导入“sentence_transformers”，需要安装该库
# 可以使用以下命令进行安装
# pip install sentence-transformers
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
# from sklearn.preprocessing import MinMaxScale

# 加载预训练模型（中文推荐'paraphrase-multilingual-MiniLM-L12-v2'）
model = SentenceTransformer('all-MiniLM-L6-v2') 
# 读取 CSV 文件
df = pd.read_csv('arxiv_data/arxiv_cv_papers.csv')
df.info()



# # 保存向量
# np.save('title_embeddings.npy', title_embeddings)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          4600 non-null   int64 
 1   title       4600 non-null   object
 2   authors     4600 non-null   object
 3   categories  4600 non-null   object
 4   published   4600 non-null   object
 5   summary     4600 non-null   object
 6   url         4600 non-null   object
dtypes: int64(1), object(6)
memory usage: 251.7+ KB


In [13]:
# 假设df是你的DataFrame对象，获取第一列（文章名字列）
article_names = df.iloc[:, 1]
article_names = article_names.tolist()
print(len(article_names))


4600


In [14]:

# 生成标题向量
title_embeddings = model.encode(article_names)  # 每个标题得到384维向量


In [15]:

print(title_embeddings.shape)
title_embedding_mean = title_embeddings.mean(axis=0)
print(title_embedding_mean.shape)

np.save('user_data/ori_recal_embedding.npy', title_embedding_mean)


(4600, 384)
(384,)


召回

In [27]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

K = 1000  # 假设这里的k为5，你可以根据实际情况修改
# 计算每个title embedding与title embedding mean的余弦相似度
similarities = cosine_similarity(title_embeddings, title_embedding_mean.reshape(1, -1)).flatten()

# 获取相似度最大的k个index
top_K_indices = np.argsort(similarities)[-K:][::-1]
# 获取df中这1000个topk indices的样本
top_K_samples = df.iloc[top_K_indices]

# for i in top_K_indices:
#     print(article_published[i])

排序

In [28]:
import pandas as pd

article_summary = top_K_samples.iloc[:, 5]
article_summary = article_summary.tolist()

summary_embeddings = model.encode(article_summary)  # 每个摘要得到384维向量
# 将 summary_embeddings 转换为 DataFrame
summary_embeddings_df = pd.DataFrame(summary_embeddings)

# 在第 8 列插入 summary_embeddings_df
top_K_samples.insert(7, 'embeddings', summary_embeddings_df.values.tolist())

top_K_samples.info()



<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1155 to 1197
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          1000 non-null   int64 
 1   title       1000 non-null   object
 2   authors     1000 non-null   object
 3   categories  1000 non-null   object
 4   published   1000 non-null   object
 5   summary     1000 non-null   object
 6   url         1000 non-null   object
 7   embeddings  1000 non-null   object
dtypes: int64(1), object(7)
memory usage: 70.3+ KB


In [29]:
import numpy as np
summary_embeddings_mean = summary_embeddings.mean(axis=0)
# 保存summary_embeddings_mean为npy文件
np.save('user_data/ori_rank_embedding.npy', summary_embeddings_mean)

sim_summary = cosine_similarity(summary_embeddings, summary_embeddings_mean.reshape(1, -1)).flatten()

k = 100

重排

In [30]:
# viewed = [410, 192, 615]
viewed = []
# 获取相似度最大的k个index
top_k_indices = np.argsort(sim_summary)[-k:][::-1]
# 可以使用布尔索引来替代列表推导式，以实现更优雅的矩阵操作。
# 首先创建一个布尔数组，标记viewed中的元素为False
mask = np.isin(np.arange(len(sim_summary)), viewed, invert=True)
# 然后结合argsort和布尔索引来筛选出不在viewed中的top_k_indices
top_k_indices = np.argsort(sim_summary[mask])[-k:][::-1]
# 由于使用了布尔索引，需要映射回原始索引
top_k_indices = np.arange(len(sim_summary))[mask][top_k_indices]
top_k_indices = [idx for idx in top_k_indices if idx not in viewed]


# 获取df中这1000个topk indices的样本
top_k_samples = top_K_samples.iloc[top_k_indices]

print(top_k_indices)

[143, 96, 50, 82, 634, 556, 647, 317, 4, 169, 749, 144, 178, 25, 562, 262, 542, 774, 175, 367, 415, 649, 198, 239, 424, 66, 406, 40, 398, 148, 494, 216, 941, 442, 692, 123, 28, 62, 0, 697, 798, 340, 846, 151, 880, 325, 308, 218, 276, 544, 255, 375, 336, 52, 970, 183, 5, 72, 97, 523, 916, 328, 610, 492, 474, 356, 240, 725, 1, 17, 19, 125, 67, 503, 21, 84, 290, 414, 452, 476, 745, 20, 532, 225, 867, 727, 948, 117, 345, 286, 912, 329, 294, 108, 29, 275, 94, 870, 219, 802]


In [31]:
print(top_k_samples)

        id                                              title  \
2616  2617  RealGeneral: Unifying Visual Generation via Te...   
2352  2353  TACO: Taming Diffusion for in-the-wild Video A...   
1729  1730  TULIP: Towards Unified Language-Image Pretraining   
3361  3362  ARMOR v0.1: Empowering Autoregressive Multimod...   
1144  1145                         Equivariant Image Modeling   
...    ...                                                ...   
380    381  JointTuner: Appearance-Motion Adaptive Joint T...   
1052  1053  Improved Alignment of Modalities in Large Visi...   
1367  1368  good4cir: Generating Detailed Synthetic Captio...   
2788  2789  ForAug: Recombining Foregrounds and Background...   
3550  3551  Unified Reward Model for Multimodal Understand...   

                                                authors  \
2616  Yijing Lin, Mengqi Huang, Shuhan Zhuang, Zhend...   
2352  Ruijie Lu, Yixin Chen, Yu Liu, Jiaxiang Tang, ...   
1729  Zineng Tang, Long Lian, Seun Eisape,

In [32]:
import pandas as pd

# 假设top_k_samples是一个列表或者DataFrame
# 如果top_k_samples是列表，将其转换为DataFrame
if not isinstance(top_k_samples, pd.DataFrame):
    top_k_samples = pd.DataFrame(top_k_samples)

# 保存为CSV文件
top_k_samples.to_csv('arxiv_data/arxiv_samples.csv', index=False)
